<a href="https://colab.research.google.com/github/vishaljoshi24/Dungeons-and-Dragons-Turn-Classification/blob/main/turn_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone 'https://github.com/vishaljoshi24/Dungeons-and-Dragons-Turn-Classification/'

# Data

In [ ]:
import pandas as pd
cleaned_df = pd.read_json('Dungeons-and-Dragons-Turn-Classification/cleaned_data.json', orient='split', compression='infer')

In [ ]:
cleaned_df

In [ ]:
!pip install dspy

In [ ]:
import dspy

In [ ]:
trainset = []

for context, current_turn in cleaned_df.values:
    trainset.append(dspy.Example(context=context, turn=current_turn).with_inputs("context", "current turn"))

In [ ]:
trainset[:5]

# Language Model

In [ ]:
lm = dspy.LM('ollama_chat/llama3.2:1b', api_base = 'http://localhost:11434', api_key='3cfbbf94908c439cb71556eaa98cd4b6.MDioHrLPBO8VqIhNLP4tfAsP')
dspy.configure(lm=lm)

# Signatures

In [ ]:
from typing import Literal


class TurnClassifier(dspy.Signature):
    data: dict[str, str] = dspy.InputField(desc = """
  {'context': 'The three previous game turns which describe a player's action or their dialogue.',
   'turn': 'The current turn taken by a player, which can include a description of an action or a piece of dialogue.',
   }""")
    behaviour_class: Literal['collaborative',
                             'contextually-relevant',
                             'goal-oriented',
                             'open-ended'] = dspy.OutputField(desc="The feature of a player's turn within a Dungeons & Dragons game: collaborative, open-ended, goal-oriented, contextually relevant")
    confidence: float = dspy.OutputField()

# Modules

In [ ]:
class ClassifyTurns(dspy.Module):
  def __init__(self):
    self.classifier = dspy.ChainOfThought(TurnClassifier, caching = False)
    #self.generator = dspy.ChainOfThought(PlayerInstruction, caching = False)

  def forward(self, data, **kwargs):
    behaviour = self.classifier(data=data).behaviour_class
    #prompt = self.generator(examples=examples).instruction
    return behaviour

In [ ]:
classify_turns = ClassifyTurns()
training_subset = trainset[:10]
behaviour_categories = []

for i in range(len(training_subset)):
  behaviour_categories.append(classify_turns(training_subset[i]))

In [ ]:
behaviour_categories